<a href="https://colab.research.google.com/github/mks999/DML_Economics/blob/main/DML_PLR_and_Feature_Inference_2_9_25_Agg_EMP_EQI_IMF_PanelMultiDiD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#1.Package Installation and Setup

# Install required packages
!pip install doubleml scikit-learn pandas numpy openpyxl matplotlib seaborn

# Import libraries
import pandas as pd
import numpy as np
from doubleml import DoubleMLData, DoubleMLPLR, DoubleMLIRM
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("=== COMPREHENSIVE DML ANALYSIS: EXPORT QUALITY & EMPLOYMENT ===")
print("Integrated PLR-IRM Framework for Multiple Specifications")
print("Comparable with GMM and Driscoll-Kraay Results")
print("=" * 70)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.8/471.8 kB 7.5 MB/s eta 0:00:00
=== COMPREHENSIVE DML ANALYSIS: EXPORT QUALITY & EMPLOYMENT ===
Integrated PLR-IRM Framework for Multiple Specifications
Comparable with GMM and Driscoll-Kraay Results


In [3]:
#2: Data Loading and Preparation

# Load data
file_path = '/content/imputed Agg. emp EQI IMF 31.8.25.xlsx'
df = pd.read_excel(file_path)

print("Data Overview:")
print(f"Shape: {df.shape}")
print(f"Countries: {df['country'].nunique()}")
print(f"Time period: {df['time'].min()}-{df['time'].max()}")
print("\nAvailable columns:")
print(df.columns.tolist())

# Basic data cleaning
df = df.sort_values(['ctryid', 'time']).reset_index(drop=True)

print(f"\nData after sorting: {df.shape}")
print("\nSample data:")
print(df.head(35))

Data Overview:
Shape: (1125, 21)
Countries: 45
Time period: 1990-2014

Available columns:
['country', 'time', 'mhtx', 'rnd', 'tariff', 'mobile', 'uv_index', 'gfcf', 'fdi', 'lnwages', 'reer', 'd1', 'aggemp', 'ctryid', 'dmy_uvindex', 'dmy_mhtx', 'msch', 'internet', 'L_aggemp', 'EQI_IMF', 'dmy_EQIimf']

Data after sorting: (1125, 21)

Sample data:
      country  time     mhtx       rnd  tariff      mobile  uv_index     gfcf  \
0   Argentina  1990  23.6244  0.472133  14.550    0.036767     40.50  13.9970   
1   Argentina  1991  23.6244  0.472133  12.383    0.075516     50.20  14.6370   
2   Argentina  1992  23.6244  0.450528  12.660    0.138792     52.40  16.7024   
3   Argentina  1993  28.9722  0.441513  12.530    0.329148     52.90  19.6881   
4   Argentina  1994  30.0064  0.436208  11.723    0.699252     57.30  19.9656   
5   Argentina  1995  30.4614  0.395236   8.340    1.160060     58.90  18.5004   
6   Argentina  1996  31.7893  0.417490  13.330    1.884800     62.70  19.6239   
7   A

In [6]:
# 3: Model Specifications Setup

# Define comprehensive model specifications
MODEL_SPECS = {
    'spec_1_gmm_nodummy': {
        'name': 'GMM Spec 1 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'reer'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 1 without dummies'
    },
    'spec_1_gmm_ctrydummy': {
        'name': 'GMM Spec 1 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'reer'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 1 with country dummies'
    },
    'spec_2_gmm_nodummy': {
        'name': 'GMM Spec 2 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'lnwages', 'reer'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 2 without dummies'
    },
    'spec_2_gmm_ctrydummy': {
        'name': 'GMM Spec 2 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'lnwages', 'reer'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 2 with country dummies'
    },
     'spec_3_gmm_nodummy': {
        'name': 'GMM Spec 3 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'msch'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 3 without dummies'
    },
    'spec_3_gmm_ctrydummy': {
        'name': 'GMM Spec 3 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'msch'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 3 with country dummies'
    },
      'spec_4_gmm_nodummy': {
        'name': 'GMM Spec 4 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 4 without dummies'
    },
    'spec_4_gmm_ctrydummy': {
        'name': 'GMM Spec 4 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 4 with country dummies'
    },
      'spec_5_gmm_nodummy': {
        'name': 'GMM Spec 5 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 5 without dummies'
    },
    'spec_5_gmm_ctrydummy': {
        'name': 'GMM Spec 5 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 5 with country dummies'
    },
      'spec_6_gmm_nodummy': {
        'name': 'GMM Spec 6 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch', 'reer'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 6 without dummies'
    },
    'spec_6_gmm_ctrydummy': {
        'name': 'GMM Spec 6 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch', 'reer'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 6 with country dummies'
    },
     'spec_7_gmm_nodummy': {
        'name': 'GMM Spec 7 (No Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch', 'reer'],
        'dummies': [],
        'description': 'Comparable to GMM Spec 7 without dummies'
    },
    'spec_7_gmm_ctrydummy': {
        'name': 'GMM Spec 7 (Country Dummies)',
        'controls': ['fdi', 'gfcf', 'rnd', 'tariff', 'internet', 'msch', 'reer'],
        'dummies': ['dmy_treatment'],
        'description': 'Comparable to GMM Spec 7 with country dummies'
    }
}


# Define analysis framework
DEPENDENT_VARS = {
    'aggemp': 'Aggregate Employment',
    # 'highskill': 'High Skill Employment', # Added based on GMM results
    # Add others when available: 'lowskill', 'semiskill'
}

TREATMENT_VARS = {
    'EQI_IMF': 'Export Quality Indicator (IMF)'
    # Add EQI_IMF when available
}

print("MODEL SPECIFICATIONS:")
print("=" * 50)
for spec_id, spec in MODEL_SPECS.items():
    print(f"{spec_id}: {spec['name']}")
    print(f"  Description: {spec['description']}")
    print(f"  Controls: {len(spec['controls'])} variables")
    print(f"  Dummies: {spec['dummies']}")
    print()

print("ANALYSIS FRAMEWORK:")
print(f"Dependent Variables: {list(DEPENDENT_VARS.keys())}")
print(f"Treatment Variables: {list(TREATMENT_VARS.keys())}")
print(f"Total Combinations: {len(DEPENDENT_VARS) * len(TREATMENT_VARS) * len(MODEL_SPECS)}")

MODEL SPECIFICATIONS:
spec_1_gmm_nodummy: GMM Spec 1 (No Dummies)
  Description: Comparable to GMM Spec 1 without dummies
  Controls: 7 variables
  Dummies: []

spec_1_gmm_ctrydummy: GMM Spec 1 (Country Dummies)
  Description: Comparable to GMM Spec 1 with country dummies
  Controls: 7 variables
  Dummies: ['dmy_treatment']

spec_2_gmm_nodummy: GMM Spec 2 (No Dummies)
  Description: Comparable to GMM Spec 2 without dummies
  Controls: 8 variables
  Dummies: []

spec_2_gmm_ctrydummy: GMM Spec 2 (Country Dummies)
  Description: Comparable to GMM Spec 2 with country dummies
  Controls: 8 variables
  Dummies: ['dmy_treatment']

spec_3_gmm_nodummy: GMM Spec 3 (No Dummies)
  Description: Comparable to GMM Spec 3 without dummies
  Controls: 5 variables
  Dummies: []

spec_3_gmm_ctrydummy: GMM Spec 3 (Country Dummies)
  Description: Comparable to GMM Spec 3 with country dummies
  Controls: 5 variables
  Dummies: ['dmy_treatment']

spec_4_gmm_nodummy: GMM Spec 4 (No Dummies)
  Description: Comp

In [7]:
# 4: Analysis Functions

def prepare_data_for_analysis(df, y_col, treatment_col, spec_name):
    """
    Prepare data for DML analysis based on specification
    """
    spec = MODEL_SPECS[spec_name]

    # Start with base controls
    x_cols = spec['controls'].copy()

    # Add dummies if specified
    if spec['dummies']:
        for dummy in spec['dummies']:
            if dummy == 'dmy_treatment':
                # Dynamically set treatment dummy
                if treatment_col == 'mhtx':
                    x_cols.append('dmy_mhtx')
                elif treatment_col == 'uv_index':
                    x_cols.append('dmy_uvindex')
                elif treatment_col == 'EQI_IMF':
                    x_cols.append('dmy_EQIimf')
            else:
                x_cols.append(dummy)

    # Select required columns
    required_cols = [y_col, treatment_col] + x_cols

    # Check available columns
    available_cols = [col for col in required_cols if col in df.columns]
    missing_cols = [col for col in required_cols if col not in df.columns]

    if missing_cols:
        print(f"Warning: Missing columns {missing_cols}")
        print(f"Available columns: {available_cols}")

    # Use available columns
    df_clean = df[available_cols].dropna()
    actual_x_cols = [col for col in x_cols if col in df.columns]

    return df_clean, actual_x_cols, len(df) - len(df_clean)

def run_plr_analysis(df_clean, y_col, treatment_col, x_cols, spec_name):
    """
    Run PLR analysis
    """
    # Create DoubleML data object
    dml_data = DoubleMLData(df_clean, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

    # Define ML models
    ml_l = GradientBoostingRegressor(n_estimators=200, max_depth=4, random_state=42)
    ml_m = RandomForestRegressor(n_estimators=200, max_depth=4, random_state=42)

    # Fit PLR model
    dml_plr = DoubleMLPLR(dml_data, ml_l=ml_l, ml_m=ml_m, n_folds=5, n_rep=3)
    dml_plr.fit(store_models=True)

    return dml_plr

def run_irm_analysis(df_clean, y_col, treatment_col, x_cols, spec_name):
    """
    Run IRM analysis
    """
    # Create DoubleML data object
    dml_data = DoubleMLData(df_clean, y_col=y_col, d_cols=treatment_col, x_cols=x_cols)

    # Define ML models (more complex for IRM)
    ml_g = GradientBoostingRegressor(n_estimators=300, max_depth=5, random_state=42)
    ml_m = RandomForestRegressor(n_estimators=300, max_depth=5, random_state=42)

    # Fit IRM model
    dml_irm = DoubleMLIRM(dml_data, ml_g=ml_g, ml_m=ml_m, n_folds=5, n_rep=3)
    dml_irm.fit(store_models=True)

    return dml_irm

print("Analysis functions defined successfully!")
print("Functions available:")
print("- prepare_data_for_analysis(): Data preparation for each specification")
print("- run_plr_analysis(): PLR model estimation")
print("- run_irm_analysis(): IRM model estimation")

Analysis functions defined successfully!
Functions available:
- prepare_data_for_analysis(): Data preparation for each specification
- run_plr_analysis(): PLR model estimation
- run_irm_analysis(): IRM model estimation


In [8]:
#5: Main Analysis Loop - PLR Results

# Initialize results storage
plr_results = {}
data_info = {}

print("=" * 70)
print("PLR ANALYSIS RESULTS")
print("=" * 70)

# Run analysis for each combination
for dep_var, dep_desc in DEPENDENT_VARS.items():
    if dep_var not in df.columns:
        print(f"Skipping {dep_var} - not available in dataset")
        continue

    for treat_var, treat_desc in TREATMENT_VARS.items():
        for spec_id, spec in MODEL_SPECS.items(): # Iterate through all new specifications

            print(f"\n{'='*50}")
            print(f"ANALYSIS: {dep_desc} ~ {treat_desc}")
            print(f"SPECIFICATION: {spec['name']}")
            print(f"{'='*50}")

            try:
                # Prepare data
                df_clean, x_cols, missing_obs = prepare_data_for_analysis(
                    df, dep_var, treat_var, spec_id
                )

                print(f"Data prepared: {len(df_clean)} obs, {missing_obs} dropped")
                print(f"Controls used: {x_cols}")

                # Run PLR
                dml_plr = run_plr_analysis(df_clean, dep_var, treat_var, x_cols, spec_id)

                # Store results
                key = f"{dep_var}_{treat_var}_{spec_id}"
                plr_results[key] = {
                    'model': dml_plr,
                    'coef': dml_plr.coef[0], # Access the scalar value
                    'se': dml_plr.se[0],     # Access the scalar value
                    'pval': dml_plr.pval[0],   # Access the scalar value
                    'ci_lower': dml_plr.coef[0] - 1.96 * dml_plr.se[0], # Access scalar values
                    'ci_upper': dml_plr.coef[0] + 1.96 * dml_plr.se[0], # Access scalar values
                    'n_obs': len(df_clean),
                    'dep_var': dep_var,
                    'treat_var': treat_var,
                    'spec_id': spec_id,
                    'controls': x_cols
                }

                data_info[key] = {
                    'df_clean': df_clean,
                    'x_cols': x_cols
                }

                # Display results - format the scalar values
                print(f"\nPLR RESULTS:")
                print(f"Coefficient: {plr_results[key]['coef']:.4f}")
                print(f"Std Error: {plr_results[key]['se']:.4f}")
                print(f"P-value: {plr_results[key]['pval']:.4f}")
                print(f"95% CI: [{plr_results[key]['ci_lower']:.4f}, {plr_results[key]['ci_upper']:.4f}]")

                significance = "***" if plr_results[key]['pval'] < 0.01 else "**" if plr_results[key]['pval'] < 0.05 else "*" if plr_results[key]['pval'] < 0.1 else ""
                print(f"Significance: {significance}")

            except Exception as e:
                print(f"Error in analysis: {str(e)}")
                continue

print(f"\n\nPLR ANALYSIS COMPLETED!")
print(f"Total successful analyses: {len(plr_results)}")

PLR ANALYSIS RESULTS

ANALYSIS: Aggregate Employment ~ Export Quality Indicator (IMF)
SPECIFICATION: GMM Spec 1 (No Dummies)
Data prepared: 1125 obs, 0 dropped
Controls used: ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'reer']

PLR RESULTS:
Coefficient: -10.0000
Std Error: 2.9531
P-value: 0.0007
95% CI: [-15.7880, -4.2120]
Significance: ***

ANALYSIS: Aggregate Employment ~ Export Quality Indicator (IMF)
SPECIFICATION: GMM Spec 1 (Country Dummies)
Data prepared: 1125 obs, 0 dropped
Controls used: ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'reer', 'dmy_EQIimf']

PLR RESULTS:
Coefficient: 2.5780
Std Error: 3.2620
P-value: 0.4293
95% CI: [-3.8155, 8.9716]
Significance: 

ANALYSIS: Aggregate Employment ~ Export Quality Indicator (IMF)
SPECIFICATION: GMM Spec 2 (No Dummies)
Data prepared: 1125 obs, 0 dropped
Controls used: ['fdi', 'gfcf', 'rnd', 'tariff', 'mobile', 'msch', 'lnwages', 'reer']

PLR RESULTS:
Coefficient: -4.6305
Std Error: 2.8138
P-value: 0.0998
95% CI: [-10.145

In [15]:
#9: Feature Importance Analysis

# Feature importance analysis for key models
print("=" * 80)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 80)

def analyze_feature_importance(model, treatment_col, x_cols, model_type):
    """Extract and display feature importance"""

    if model.models is not None:
        try:
            if model_type == 'PLR':
                fitted_ml_l = model.models['ml_l'][treatment_col][0][0]
                fitted_ml_m = model.models['ml_m'][treatment_col][0][0]

                print(f"\n{model_type} FEATURE IMPORTANCE:")

                # Outcome model (ml_l)
                if hasattr(fitted_ml_l, 'feature_importances_'):
                    # Ensure correct feature names are used based on x_cols and treatment_col
                    outcome_features = x_cols + [treatment_col]
                    if len(fitted_ml_l.feature_importances_) == len(outcome_features):
                        importance_l = pd.Series(
                            fitted_ml_l.feature_importances_,
                            index=outcome_features
                        ).sort_values(ascending=False)

                        print(f"\nOutcome Model (Employment Prediction):")
                        for var, imp in importance_l.head(5).items():
                            print(f"  {var}: {imp:.4f}")

                # Treatment model (ml_m)
                if hasattr(fitted_ml_m, 'feature_importances_'):
                     if len(fitted_ml_m.feature_importances_) == len(x_cols):
                        importance_m = pd.Series(
                            fitted_ml_m.feature_importances_,
                            index=x_cols
                        ).sort_values(ascending=False)

                        print(f"\nTreatment Model (Export Quality Prediction):")
                        for var, imp in importance_m.head(5).items():
                            print(f"  {var}: {imp:.4f}")

            elif model_type == 'IRM':
                fitted_ml_g = model.models['ml_g'][treatment_col][0][0]
                fitted_ml_m = model.models['ml_m'][treatment_col][0][0]

                print(f"\n{model_type} FEATURE IMPORTANCE:")

                # Outcome model with interactions (ml_g)
                if hasattr(fitted_ml_g, 'feature_importances_'):
                    # Ensure correct feature names are used based on x_cols and treatment_col
                    outcome_features = x_cols + [treatment_col]
                    if len(fitted_ml_g.feature_importances_) == len(outcome_features):
                        importance_g = pd.Series(
                            fitted_ml_g.feature_importances_,
                            index=outcome_features
                        ).sort_values(ascending=False)

                        print(f"\nOutcome Model with Interactions (ml_g):")
                        for var, imp in importance_g.head(5).items():
                            print(f"  {var}: {imp:.4f}")

                # Treatment model (ml_m) - same as PLR
                if hasattr(fitted_ml_m, 'feature_importances_'):
                     if len(fitted_ml_m.feature_importances_) == len(x_cols):
                        importance_m = pd.Series(
                            fitted_ml_m.feature_importances_,
                            index=x_cols
                        ).sort_values(ascending=False)

                        print(f"\nTreatment Model (Export Quality Prediction):")
                        for var, imp in importance_m.head(5).items():
                            print(f"  {var}: {imp:.4f}")

        except Exception as e:
            print(f"Feature importance extraction failed: {e}")

# Analyze feature importance for selected models
selected_analyses = list(plr_results.keys())[:2]  # Analyze first 2 models

for key in selected_analyses:
    plr = plr_results[key]
    # irm = irm_results.get(key) # Remove IRM

    print(f"\n{'='*60}")
    print(f"FEATURE IMPORTANCE: {plr['dep_var']} ~ {plr['treat_var']}")
    print(f"SPECIFICATION: {MODEL_SPECS[plr['spec_id']]['name']}")
    print(f"{'='*60}")

    # PLR feature importance
    analyze_feature_importance(plr['model'], plr['treat_var'], plr['controls'], 'PLR')

    # IRM feature importance
    # if irm: # Remove IRM
    #     analyze_feature_importance(irm['model'], irm['treat_var'], irm['controls'], 'IRM')

FEATURE IMPORTANCE ANALYSIS

FEATURE IMPORTANCE: aggemp ~ EQI_IMF
SPECIFICATION: GMM Spec 1 (No Dummies)

PLR FEATURE IMPORTANCE:

Treatment Model (Export Quality Prediction):
  rnd: 0.6530
  tariff: 0.1453
  mobile: 0.0774
  reer: 0.0499
  msch: 0.0318

FEATURE IMPORTANCE: aggemp ~ EQI_IMF
SPECIFICATION: GMM Spec 1 (Country Dummies)

PLR FEATURE IMPORTANCE:

Treatment Model (Export Quality Prediction):
  dmy_EQIimf: 0.7252
  rnd: 0.1871
  fdi: 0.0392
  reer: 0.0157
  tariff: 0.0155


In [16]:
#10: Economic Interpretation and Policy Implications

# Economic interpretation and policy implications
print("=" * 80)
print("ECONOMIC INTERPRETATION AND POLICY IMPLICATIONS")
print("=" * 80)

def interpret_results(coef, treat_var, method):
    """Provide economic interpretation of results"""

    if treat_var == 'EQI_IMF':
        treatment_name = "Export Quality Indicator (IMF)"
        unit = "unit increase"
    elif treat_var == 'uv_index':
        treatment_name = "export unit value (quality)"
        unit = "unit increase in quality index"

    effect_direction = "increases" if coef > 0 else "decreases"
    effect_magnitude = "large" if abs(coef) > 0.1 else "moderate" if abs(coef) > 0.05 else "small"

    interpretation = f"""
    A 1-{unit} in {treatment_name} {effect_direction} employment by {abs(coef):.4f} percentage points.
    This represents a {effect_magnitude} {effect_direction[:-1]}ing effect estimated using {method}.
    """

    return interpretation.strip()

# Interpret key results
print("\nKEY FINDINGS:")
print("-" * 40)

for i, (key, plr) in enumerate(plr_results.items()):
    if i >= 3:  # Limit to first 3 results
        break

    # irm = irm_results.get(key) # Remove IRM

    dep_desc = DEPENDENT_VARS[plr['dep_var']]
    treat_desc = TREATMENT_VARS[plr['treat_var']]
    spec_name = MODEL_SPECS[plr['spec_id']]['name']

    print(f"\n{i+1}. {dep_desc} and {treat_desc} ({spec_name})")
    print("   " + "="*50)

    # PLR interpretation
    print(f"   PLR Result: {interpret_results(plr['coef'], plr['treat_var'], 'PLR')}")

    # IRM interpretation if available - Remove IRM interpretation
    # if irm:
    #     print(f"   IRM Result: {interpret_results(irm['coef'], irm['treat_var'], 'IRM')}")

    #     # Compare methods - Remove IRM comparison
    #     if abs(plr['coef'] - irm['coef']) / abs(plr['coef']) > 0.1:
    #         print(f"   → HETEROGENEITY: Significant differences suggest country-specific effects")
    #     else:
    #         print(f"   → CONSISTENCY: PLR and IRM estimates are similar")

print(f"\n\nPOLICY IMPLICATIONS:")
print("-" * 30)
print("1. Export Quality Upgrading:")
print("   - Positive coefficients suggest export quality improvements boost employment")
print("   - Effect sizes indicate magnitude of policy impact")

print("\n2. Technology Export Promotion:")
print("   - MHTX coefficients show employment effects of high-tech export policies")
print("   - Consider heterogeneous effects across countries (IRM insights)")

print("\n3. Targeted Interventions:")
print("   - IRM results reveal which countries benefit most from quality upgrading")
print("   - PLR provides average effects for broad policy recommendations")

print("\n4. Robustness:")
print("   - Compare with GMM and Driscoll-Kraay results for validation")
print("   - Multiple specifications ensure result stability")

ECONOMIC INTERPRETATION AND POLICY IMPLICATIONS

KEY FINDINGS:
----------------------------------------

1. Aggregate Employment and Export Quality Indicator (IMF) (GMM Spec 1 (No Dummies))
   PLR Result: A 1-unit increase in Export Quality Indicator (IMF) decreases employment by 10.0000 percentage points.
    This represents a large decreaseing effect estimated using PLR.

2. Aggregate Employment and Export Quality Indicator (IMF) (GMM Spec 1 (Country Dummies))
   PLR Result: A 1-unit increase in Export Quality Indicator (IMF) increases employment by 2.5780 percentage points.
    This represents a large increaseing effect estimated using PLR.

3. Aggregate Employment and Export Quality Indicator (IMF) (GMM Spec 2 (No Dummies))
   PLR Result: A 1-unit increase in Export Quality Indicator (IMF) decreases employment by 4.6305 percentage points.
    This represents a large decreaseing effect estimated using PLR.


POLICY IMPLICATIONS:
------------------------------
1. Export Quality Upgra